## **Workflow -  Mintpy Smallbaseline** 

- Original author: Yunjun Zhang, [MintPy-totorial](https://github.com/insarlab/MintPy-tutorial.git)

- Modified by:  Xuanxuan Wang

- Date:  Oct 2025

<b> 1. Create a template </b>

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import mintpy.cli.view as view
import mintpy.cli.info as info
import mintpy.cli.smallbaselineApp as smallbaselineApp
mintpy_workspace="/mnt/e/InSAR/Yongzhou/GAMMA/mintpy_1"
os.makedirs(mintpy_workspace, exist_ok=True)
template_name = "TYSAR"

In [ ]:
reference_point_xy = [71, 192]
if reference_point_xy and '-' not in reference_point_xy:
    ref_point_comment = f"#mintpy.reference.yx = {int(reference_point_xy[1])}, {int(reference_point_xy[0])}"
else:
    ref_point_comment = f"mintpy.reference.yx = auto"

In [ ]:
mintpy_network_mincoh = 0.19
mintpy_networkInv_minTcoh = 0.5

# ISCE_template
# template_content = f"""# vim: set filetype=cfg:
# ##------------------------------- ISCE/stripmapStack OPTIONS ------------------##
# isce.processor          = stripmapStack  #[stripmapStack, topsStack]
# isce.demSNWE            = 22, 24, 113, 115  #[S, N, W, E] in degree
# isce.demFile            = /mnt/g/Shenzhen_ty/DEM/demLat_N22_N24_Lon_E113_E115.dem.wgs84
# isce.azimuthLooks       = 3
# isce.rangeLooks         = 3
# #isce.maxTempBaseline    = 35.0
# #isce.maxPerpBaseline    = 150.0
# isce.unwrapMethod       = snaphu
# isce.filtStrength       = 0.8
# isce.applyWaterMask     = yes


# ##------------------------------- mintpy OPTIONS -------------------------------##
# ## load_data.py -H to check more details and example inputs.
# mintpy.load.processor        = isce
# ##---------for ISCE only:
# mintpy.load.metaFile         = ../merged/SLC/20240120/referenceShelve/data.dat
# mintpy.load.baselineDir      = ../baselines
# ##---------interferogram datasets:
# mintpy.load.unwFile          = ../Igrams/*/filt_*.unw
# mintpy.load.corFile          = ../Igrams/*/filt_*.cor
# mintpy.load.connCompFile     = ../Igrams/*/filt_*.unw.conncomp
# ##---------geometry datasets:
# mintpy.load.demFile          = ../geom_reference/hgt.rdr
# mintpy.load.lookupYFile      = ../geom_reference/lat.rdr
# mintpy.load.lookupXFile      = ../geom_reference/lon.rdr
# mintpy.load.incAngleFile     = ../geom_reference/los.rdr
# mintpy.load.azAngleFile      = ../geom_reference/los.rdr
# mintpy.load.shadowMaskFile   = ../geom_reference/shadowMask.rdr
# mintpy.load.waterMaskFile    = ../geom_reference/waterMask.rdr
# mintpy.load.bperpFile        = None

# mintpy.network.coherenceBased   = yes
# mintpy.network.keepMinSpanTree  = no
# mintpy.network.minCoherence     = 0.35
# mintpy.troposphericDelay.weatherModel = ERA5
# mintpy.deramp	                = linear

# ##------------------------------- HDF-EOS 5 metadata ———————————————————————————##
# ORBIT_DIRECTION = DESCENDING
# trackNumber     = 1



# """
template_content = f"""# vim: set filetype=cfg:
##------------------------  MintPy ----------------------------------##
########## 1. load_data
mintpy.load.processor      = gamma
mintpy.load.unwFile        = ../interferograms/*/*rlks.adf.unw
mintpy.load.corFile        = ../interferograms/*/*rlks.adf.cor
mintpy.load.connCompFile   = None
mintpy.load.intFile        = None

mintpy.load.demFile        = ../geometry/*.rdc.dem
mintpy.load.lookupYFile    = ../geometry/*.UTM_TO_RDC
mintpy.load.lookupXFile    = ../geometry/*.UTM_TO_RDC
mintpy.load.incAngleFile   = None
mintpy.load.azAngleFile    = None
mintpy.load.shadowMaskFile = None
mintpy.load.waterMaskFile  = None

########## 2. modify_network
mintpy.network.coherenceBased       = yes       #[yes / no], auto for yes, exclude interferograms with coherence < minCoherence
mintpy.network.keepMinSpanTree      = yes
mintpy.network.minCoherence    = {mintpy_network_mincoh}  #[0.0-1.0], auto for 0.7

########## 3. reference_point
{ref_point_comment}

########## 4. correct_unwrap_error (optional)
mintpy.unwrapError.method          = auto  #[bridging / phase_closure / bridging+phase_closure / no], auto for no
mintpy.unwrapError.waterMaskFile   = auto  #[waterMask.h5 / no], auto for waterMask.h5 or no [if not found]
mintpy.unwrapError.connCompMinArea = auto


########## 5. invert_network
mintpy.networkInversion.minTempCoh  = {mintpy_networkInv_minTcoh} #[0.0-1.0], auto for 0.7, min temporal coherence for mask


########## 8. correct_troposphere (optional but recommended)
mintpy.troposphericDelay.method = height_correlation #[pyaps / height_correlation / gacos / no], auto for pyaps

########## 9. deramp (optional)
mintpy.deramp          = linear  #[no / linear / quadratic], auto for no - no ramp will be removed

########## 11.2 reference_date
## Reference all time-series to one date in time
## reference: Yunjun et al. (2019, section 4.9)
## no     - do not change the default reference date (1st date)
mintpy.reference.date =  no #[reference_date.txt / 20090214 / no], auto for reference_date.txt

"""

with open(f'{mintpy_workspace}/{template_name}.txt', "w", encoding ='utf-8') as f:
    f.write(template_content)
print(f"Template file created at: {f'{mintpy_workspace}/{template_name}.txt'}")

<b> 2. Load the data </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "load_data", '--dir', mintpy_workspace])

In [ ]:
info.main([f'{mintpy_workspace}/inputs/ifgramStack.h5'])

In [ ]:
# ft_int = glob.glob(os.path.join(os.path.dirname(mintpy_workspace),'Igrams','*','filt_*.int'), recursive=True)
# pairs = sorted([os.path.basename(os.path.dirname(ft)) for ft in ft_int])
# print(f"Pairs found: {pairs}")
# os.makedirs(f'{mintpy_workspace}/pic/ifgs', exist_ok=True)
# for pair in pairs:
#     view.main([f'{mintpy_workspace}/inputs/ifgramStack.h5', pair, '--ncols', '3', '-o', f'{mintpy_workspace}/pic/ifgs/ifgramStack_{pair}.png'])

<b> 3. Network modification </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "modify_network", '--dir', mintpy_workspace])

In [ ]:
import matplotlib
%matplotlib inline

In [ ]:
import mintpy.cli.plot_network
mintpy.cli.plot_network.main([f'{mintpy_workspace}/inputs/ifgramStack.h5', '--save'])

<b> 4. Select reference point </b>

><span style="font-size: 0.8em;">Default: manually select the reference point, which requires a renewal reference_point.py to be built in mintpy library.</span>

><span style="font-size: 0.8em;">Comment the line `"--method", "manual"` to use the old method for reference point selection.</span>

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
import mintpy.cli.generate_mask
mintpy.cli.generate_mask.main([
    f'{mintpy_workspace}/inputs/ifgramStack.h5',
    '--nonzero',
    '-o',
    f'{mintpy_workspace}/maskConnComp.h5',
    '--update'
    ])
import mintpy.cli.temporal_average
mintpy.cli.temporal_average.main([
    f'{mintpy_workspace}/inputs/ifgramStack.h5',
    '--dataset',
    'coherence',
    '-o',
    f'{mintpy_workspace}/avgSpatialCoh.h5',
    '--update'
    ])
import mintpy.cli.reference_point
mintpy.cli.reference_point.main([
    f'{mintpy_workspace}/inputs/ifgramStack.h5', 
    '-t', 
    f'{mintpy_workspace}/{template_name}.txt',
    '-c',
    f'{mintpy_workspace}/avgSpatialCoh.h5',
    '-y',
    str(reference_point_xy[1]),
    '-x',
    str(reference_point_xy[0])
    # '--method',
    # 'manual'
    ])

<b> 5. Quick overview </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "quick_overview", '--dir', mintpy_workspace])

In [ ]:
import matplotlib
%matplotlib inline

In [ ]:
view.main([f'{mintpy_workspace}/avgPhaseVelocity.h5', '-o', f'{mintpy_workspace}/pic/avgPhaseVelocity.png'])

<b> 6. (Optional) Phase-unwrapping error correction </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "correct_unwrap_error", '--dir', mintpy_workspace])

<b> 7. Network inversion </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "invert_network", '--dir', mintpy_workspace])

In [ ]:
view.main([f'{mintpy_workspace}/timeseries.h5', '--wrap', '--wrap-range', '-5', '5', '-u', 'cm', 
'--notick','--noaxis', '-o', f'{mintpy_workspace}/pic/timeseries_wrap5.png'])

In [ ]:
view.main([f'{mintpy_workspace}/temporalCoherence.h5', '-c', 'gray', '--notick', '--noaxis', '--noverbose', '-o', f'{mintpy_workspace}/pic/temporalCoherence.png'])
view.main([f'{mintpy_workspace}/maskTempCoh.h5', '-c', 'gray', '--notick', '--noaxis', '--noverbose', f'-o', f'{mintpy_workspace}/pic/maskTempCoh.png'])

<b> 8. Troperspheric delay correction </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "correct_troposphere", '--dir', mintpy_workspace])

In [ ]:
TDC_suffix = ['ERA5', 'tropHgt']
for suffix in TDC_suffix:
    h5_tc_temp = glob.glob(os.path.join(mintpy_workspace, f'timeseries_{suffix}.h5'))
    if h5_tc_temp:
        h5_tc = h5_tc_temp[0]
if h5_tc:
    h5_tc_name = os.path.basename(h5_tc).replace('.h5','')
    view.main([h5_tc, '--wrap', '--wrap-range', '-5', '5', '--notick','--noaxis', '-o', f'{mintpy_workspace}/pic/{h5_tc_name}_wrap5.png'])

<b> 9. Phase deramping </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "deramp", '--dir', mintpy_workspace])

In [ ]:
h5_deramp = glob.glob(os.path.join(mintpy_workspace, "*_ramp.h5"))[0]
if h5_deramp:
    h5_deramp_name = os.path.basename(h5_deramp).replace('.h5','')
    view.main([h5_deramp, '--wrap', '--wrap-range', '-5', '5', '--notick','--noaxis', '-o', f'{mintpy_workspace}/pic/{h5_deramp_name}_wrap5.png'])

<b> 10. Topographic residual (DEM error) correction </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "correct_topography", '--dir', mintpy_workspace])

In [ ]:
h5_demerr = glob.glob(os.path.join(mintpy_workspace, "*_demErr.h5"))[0]
if h5_demerr:
    h5_demerr_name = os.path.basename(h5_demerr).replace('.h5', '')
    view.main([h5_demerr, '--wrap', '--wrap-range', '-5', '5', '--notick','--noaxis', '-o', f'{mintpy_workspace}/pic/{h5_demerr_name}_wrap5.png'])
# view.main([f'{mintpy_workspace}/timeseries_ERA5_demErr.h5', '--wrap', '--wrap-range', '-5', '5', '--notick','--noaxis', '-o', f'{mintpy_workspace}/pic/timeseries_ERA5_ramp_demErr_wrap5.png'])

<b> 11. Phase residual RMS for noise evaluation </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "residual_RMS", '--dir', mintpy_workspace])

<b> 12. Change reference date </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "reference_date", '--dir', mintpy_workspace])

<b> 13. Velocity </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "velocity", '--dir', mintpy_workspace])

In [ ]:
info.main([f'{mintpy_workspace}/velocity.h5'])

In [ ]:
h5_vel = glob.glob(os.path.join(mintpy_workspace, "velocity*.h5"))
for h5 in h5_vel:
    h5_name = os.path.basename(h5).replace('.h5', '')
    view.main([h5, '--notick','--noaxis', '--noverbose', '-o', f'{mintpy_workspace}/pic/{h5_name}.png'])

<b> 14. Geocode </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "geocode", '--dir', mintpy_workspace])

In [ ]:
view.main([f'{mintpy_workspace}/geo/geo_velocity.h5', 'velocity', '--dem', f'{mintpy_workspace}/geo/geo_geometryRadar.h5', '--shade-exag', '0.05', '--figsize', '12', '12', '--notitle', '--notick', '--noaxis', '-o', f'{mintpy_workspace}/pic/geo_velocity.png'])

<b> 15. Output to Google Earth format </b>

In [ ]:
smallbaselineApp.main([f'{mintpy_workspace}/{template_name}.txt', "--dostep", "google_earth", '--dir', mintpy_workspace])

<b> 16. Organize output results </b>

In [ ]:
kmz_files = glob.glob(os.path.join(mintpy_workspace, 'geo', '*.kmz'))
for kmz in kmz_files:
    os.rename(kmz, os.path.join(mintpy_workspace, 'pic', os.path.basename(kmz)))
pdf_files = glob.glob(os.path.join(mintpy_workspace, '*.pdf'))
for pdf in pdf_files:
    os.rename(pdf, os.path.join(mintpy_workspace, 'pic', os.path.basename(pdf)))
png_files= glob.glob(os.path.join(mintpy_workspace, '*.png'))
for png in png_files:
    os.rename(png, os.path.join(mintpy_workspace, 'pic', os.path.basename(png)))

<b> 17. Save to QGIS format </b>

In [ ]:
ts_file = glob.glob(os.path.join(mintpy_workspace, 'geo', 'geo_timeseries*.h5'))[0]
geom_file = os.path.join(mintpy_workspace, 'geo', 'geo_geometryRadar.h5')
output_ts = ts_file.replace('.h5', '.shp')
import mintpy.cli.save_qgis as save_qgis
save_qgis.main([ts_file, '-g', geom_file, '-o', output_ts])